# Analysis of a Single Shot - Errors on the IV Characteristic 

This is a notebook for analysing a single shot from second round magnum data and looking closely at the distribution of values around the averaged IV characteristic. There is also a section at the end for looking at the distribution of points around the single bias (i.e. $J_{sat}$) and a comparison of this to the error distribution. 

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr 
import scipy.stats as stat
import sys
import os
import glob
import re
import importlib
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.core.constants as c
import flopter.magnum.database as ut
import flopter.core.fitters as fts
import flopter.magnum.utils as mgut

In [2]:
# Create analysed dataset metadata 

path_to_datasets = '/home/jleland/data/external/magnum/'
# path_to_analysed_datasets = 'analysed_4_downsampled/*nc'
path_to_analysed_datasets = 'analysed_4_downsampled'
os.chdir(path_to_datasets)


analysed_infos_df = mgut.get_dataset_metadata(path_to_analysed_datasets)
# analysed_dataset_fns = glob.glob(path_to_analysed_datasets)
# analysed_dataset_fns.sort()

# analysed_infos = []

# for i, anal_ds in enumerate(analysed_dataset_fns):
#     match = re.search("\/a{1}([0-9]{3})_([0-9]{3})_([0-9]{19})\.nc", anal_ds)
#     shot_index = int(match.group(1))
#     adc_index = int(match.group(2))
#     shot_timestamp = int(match.group(3))
#     shot_time = ut.human_datetime_str(int(match.group(3)))
    
#     ds = xr.open_dataset(anal_ds)
#     time_len = len(ds['time'])
#     sweep_len = len(ds['sweep'])
    
#     analysed_infos.append({
#         'adc_index':adc_index, 
#         'shot_number':shot_index, 
#         'shot_timestamp':shot_timestamp, 
#         'shot_time':shot_time, 
#         'filename':anal_ds,
#         'time_len': time_len,
#         'sweep_len': sweep_len
#     })

# analysed_infos_df = pd.DataFrame(analysed_infos).set_index('adc_index')

In [3]:
adc_index = 329

In [4]:
# shot_number = 337
shot_number = 338

In [5]:
single_bias_shot = 112
# single_bias_shot = 111

In [6]:
# analysed_infos_df.loc[adc_index]
analysed_infos_df

,shot_number,shot_timestamp,shot_time,filename,time_len,sweep_len
adc_index,,,,,,
5,3,6696074692831397888,2019-05-28 14:49:40,analysed_4_downsampled/a003_005_66960746928313...,50,499
6,4,6696076438102672384,2019-05-28 14:56:27,analysed_4_downsampled/a004_006_66960764381026...,50,798
7,5,6696076936923400192,2019-05-28 14:58:23,analysed_4_downsampled/a005_007_66960769369234...,50,798
8,6,6696082451310700544,2019-05-28 15:19:47,analysed_4_downsampled/a006_008_66960824513107...,50,799
9,7,6696082985479260160,2019-05-28 15:21:51,analysed_4_downsampled/a007_009_66960829854792...,50,1999
...,...,...,...,...,...,...
522,518,6699789649312836608,2019-06-07 15:05:36,analysed_4_downsampled/a518_522_66997896493128...,50,999
523,519,6699789906626051072,2019-06-07 15:06:36,analysed_4_downsampled/a519_523_66997899066260...,50,997
524,520,6699790213374673920,2019-06-07 15:07:47,analysed_4_downsampled/a520_524_66997902133746...,50,999


In [7]:
# Get row from shot_number instead of ADC index
shot_oi = analysed_infos_df.loc[analysed_infos_df['shot_number'] == shot_number]
shot_oi['filename'].values

array(['analysed_4_downsampled/a338_330_6699049326982551552.nc'],
      dtype=object)

In [8]:
analysis_ds = xr.open_dataset(shot_oi['filename'].values[0])
metadata_ds = xr.open_dataset('all_meta_data.nc').sel(shot_number=shot_number).load()
print(analysis_ds)
metadata_ds

<xarray.Dataset>
Dimensions:     (direction: 2, probe: 2, sweep: 2199, time: 50)
Coordinates:
  * time        (time) float64 0.0 0.0001 0.0002 0.0003 ... 0.0047 0.0048 0.0049
  * direction   (direction) object 'up' 'down'
  * probe       (probe) object 'S' 'L'
Dimensions without coordinates: sweep
Data variables:
    voltage     (probe, direction, sweep, time) float64 ...
    current     (probe, direction, sweep, time) float64 ...
    shot_time   (probe, direction, sweep, time) float64 ...
    start_time  (probe, direction, sweep) float64 ...


<xarray.Dataset>
Dimensions:                 (ts_radial_pos: 46)
Coordinates:
    shot_number             int32 338
    ts_number               float64 407.0
    ts_timestamp            float64 6.699e+18
    ts_time                 datetime64[ns] 2019-06-05T15:12:46.670370
    adc_index               float64 330.0
    adc_time                datetime64[ns] 2019-06-05T15:12:46
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            object '2019-06-05 16h 13m 08s TT_06699049326982551963.adc'
    ts_density              (ts_radial_pos) float64 nan nan nan ... nan nan nan
    ts_temperature          (ts_radial_pos) float64 nan nan nan ... nan nan nan
    ts_d_density            (ts_radial_pos) float64 nan nan nan ... nan nan nan
    ts_d_temperature        (ts_radial_pos) float64 nan nan nan ... nan nan nan
    adc_folder              object '2019-06-05_Leland/'
    adc_calibration_index   object '272'
    adc_4_probe             object 'S'
    adc_5_probe             object 'L'
    ts_temp_max             float64 4.816
    ts_dens_max             float64 1.679e+19
    shot_end_time           datetime64[ns] 2019-06-05T15:13:13
    ts_fl                   bool True
    shot_time               datetime64[ns] 2019-06-05T15:12:42
    shot_source_current     float64 110.1
    shot_b_field            float64 1.2
    shot_tilt               float64 8.991
    shot_target_pos         float64 -0.15
    shot_target_rot         float64 -89.94
    shot_target_pumpspeed   int32 82
    shot_source_pumpspeed   int32 82
    shot_heating_pumpspeed  int32 82
    shot_source_pos         float64 1.2
    shot_deuterium_gf       float64 0.003443
    shot_hydrogen_gf        float64 8.442
    shot_helium_gf          float64 0.0
    shot_hydrogen_puffed    float64 -0.01433
    shot_helium_puffed      float64 -0.0523
    adc_timestamp           float64 6.699e+18
    adc_4_coax              float64 1.0
    adc_4_shunt_resistance  float64 1.0
    adc_5_coax              float64 2.0
    adc_5_shunt_resistance  float64 1.0
    adc_calibration_fl      float64 0.0
    shot_fl                 float64 1.0
    acquisition_length      float64 22.0
    adc_end_time            datetime64[ns] 2019-06-05T15:13:08
    adc_end_timestamp       float64 6.699e+18
    adc_freqs               float64 1e+06

In [9]:
all_md_ds = xr.open_dataset('all_meta_data.nc')
same_param_ds = all_md_ds.where(np.logical_and(
    np.logical_and(all_md_ds.shot_b_field.round(1) == 1.2, all_md_ds.shot_source_current.round() == 110.0),
    np.logical_and(all_md_ds.shot_hydrogen_gf >= 8.35, all_md_ds.shot_hydrogen_gf <= 8.65)
), drop=True)
# all_md_ds.where(all_md_ds.shot_b_field.round(1) == 1.2, drop=True)
# all_md_ds.where(np.logical_and(all_md_ds.shot_hydrogen_gf >= 8.35, all_md_ds.shot_hydrogen_gf <= 8.65), drop=True)
# all_md_ds.where(all_md_ds.shot_source_current.round() == 110.0, drop=True)

In [10]:
probe = 'S'

magnum_probes = lp.MagnumProbes()

gaussian = fts.NormalisedGaussianFitter()

In [11]:
all_ts_ds = xr.open_dataset('full_ts_dataset.nc').swap_dims({'ts_time':'ts_number'})
all_ts_ds

<xarray.Dataset>
Dimensions:           (ts_number: 627, ts_radial_pos: 46)
Coordinates:
  * ts_radial_pos     (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
  * ts_number         (ts_number) int64 0 1 2 3 4 5 ... 621 622 623 624 625 626
    ts_timestamp      (ts_number) int64 ...
    ts_time           (ts_number) datetime64[ns] 2019-05-28T14:49:40.686322 ... 2019-06-07T15:10:32.317018
    shot_number       (ts_number) int64 ...
Data variables:
    ts_density        (ts_number, ts_radial_pos) float64 ...
    ts_d_density      (ts_number, ts_radial_pos) float64 ...
    ts_temperature    (ts_number, ts_radial_pos) float64 ...
    ts_d_temperature  (ts_number, ts_radial_pos) float64 ...

## Do some analysis of shot

In [12]:
fig, ax = plt.subplots(3, sharex=True)
shot_ds = analysis_ds.sel(probe=probe)

for sweep in shot_ds['sweep'].values[::2]:
    sweep_ds = shot_ds.sel(sweep=sweep, direction='up')
    ax[0].plot(sweep_ds['voltage'].values, sweep_ds['current'].values, color='silver')
    sweep_ds = shot_ds.sel(sweep=sweep, direction='down')
    ax[1].plot(sweep_ds['voltage'].values, sweep_ds['current'].values, color='silver')
    
sweep_avg_ds = shot_ds.mean('sweep')
for i, direction in enumerate(['up', 'down']):
    ax[i].plot(sweep_avg_ds.sel(direction=direction)['voltage'].values, sweep_avg_ds.sel(direction=direction)['current'].values, 'k', label='Average')
    ax[2].plot(sweep_avg_ds.sel(direction=direction)['voltage'].values, sweep_avg_ds.sel(direction=direction)['current'].values, label=f'{direction} avg.')
plt.show()

In [13]:
fig, ax = plt.subplots(2, sharex=True)
shot_ds = analysis_ds.sel(probe=probe) #.mean('direction')

for sweep in shot_ds['sweep'].values[::4]:
    sweep_ds = shot_ds.sel(sweep=sweep)
    ax[0].plot(sweep_ds['voltage'].values, sweep_ds['current'].values, color='silver', zorder=-2)

    
sweep_avg_ds = shot_ds.mean(['sweep', 'direction'])
sweep_avg_ds = sweep_avg_ds.assign({'d_current': shot_ds.std(['sweep', 'direction'])['current'], 
                                    'd_voltage': shot_ds.std(['sweep', 'direction'])['voltage']}) 


# (np.abs(sweep_avg_ds['current']) / sweep_avg_ds.set_coords('voltage')['d_current']).plot.line(x='voltage', ax=ax[1])
sweep_avg_ds.set_coords('voltage')['d_current'].plot.line(x='voltage', ax=ax[1])
ax[1].set_title('')

ax[0].errorbar(sweep_avg_ds['voltage'].values, sweep_avg_ds['current'].values, 
               yerr=sweep_avg_ds['d_current'].values,
               xerr=sweep_avg_ds['d_voltage'].values,
               linestyle='none', color='k', ecolor='k', label='Averaged IV', zorder=2)
ax[0].set_ylabel(r'$I$ (A)')
ax[1].set_ylabel(r'$\Delta I$ (A)')
ax[1].set_xlabel(r'$V$ (A)')
    
ax[0].legend()
plt.show()

In [14]:
# Plot looking at Gaussian nature of distribution of points around mean

zeroed_ds = shot_ds - sweep_avg_ds
bins = np.linspace(-0.06,0.06,200)

select_sweeps = [23,-1]

fig, axes = plt.subplots(2, sharex=True)

for i, time in enumerate(zeroed_ds['time'].values[select_sweeps]):
    time_ds = zeroed_ds.sel(time=time)
#     bins_time = bins + sweep_avg_ds.sel(time=time, direction='up')['current'].values
#     print(np.mean(bins), np.mean(bins_time))
    hist, bin_edges = np.histogram(time_ds['current'].values / sweep_avg_ds.sel(time=time)['current'].values, 
                                   bins='auto', density=True)
#     hist, bin_edges = np.histogram(time_ds['current'].values, bins='auto', density=True)
    bins = (bin_edges[:-1] + bin_edges[1:]) / 2
#     axes[i].plot(bins, hist, label=f"V={sweep_avg_ds.sel(time=time).mean('direction')['voltage'].values}")
    axes[i].plot(bins, hist, label=r"V={:.3g}, std = {:.3g}".format(sweep_avg_ds.sel(time=time)['voltage'].values, sweep_avg_ds.sel(time=time)['d_current'].values))

    fit_data = gaussian.fit(bins, hist) 
    
    axes[i].plot(*fit_data.get_fit_plottables(), label=r'Gaussian fit - $\sigma$ = {:.3g}'.format(fit_data.get_param("sigma")))
    
    
#     time_ds = zeroed_ds.sel(time=time, direction='down')
#     hist = time_ds['current'].plot.hist(bins=bins)
axes[0].legend()
axes[1].legend()
plt.show()

In [17]:
import flopter.core.colours as col
probe_colours = {
    'L': col.palettes['c'][3],
    'S': col.palettes['c'][2],
    'B': col.palettes['c'][1],
    'R': col.palettes['b'][2],
}

In [31]:
# Figure combining above 2 plots
fig, ax = plt.subplots(2, figsize=[8, 6])
shot_ds = analysis_ds.sel(probe=probe) #.mean('direction')

for sweep in shot_ds['sweep'].values[::50]:
    sweep_ds = shot_ds.sel(sweep=sweep)
    ax[0].plot(sweep_ds['voltage'].values, sweep_ds['current'].values, 
               linestyle='none', marker='.', color='silver', 
               zorder=-2, alpha=0.3)

    
sweep_avg_ds = shot_ds.mean(['sweep', 'direction'])
sweep_avg_ds = sweep_avg_ds.assign({'d_current': shot_ds.std(['sweep', 'direction'])['current'], 
                                    'd_voltage': shot_ds.std(['sweep', 'direction'])['voltage']}) 

ax[0].errorbar(sweep_avg_ds['voltage'].values, sweep_avg_ds['current'].values, 
               yerr=sweep_avg_ds['d_current'].values,
#                xerr=sweep_avg_ds['d_voltage'].values,
               linestyle='none', color=probe_colours[probe], 
               ecolor=probe_colours[probe], marker='o', mfc='none',
               label=r'Sweep-Averaged IV ($\bar{I}$)', zorder=2)
ax[0].set_ylabel(r'Probe Current [A]')    
ax[0].set_xlabel(r'Probe Voltage [V]')
ax[0].legend()

select_sweeps = [-1]
zeroed_ds = shot_ds - sweep_avg_ds
bins = np.linspace(-0.4,0.4,200)

for i, time in enumerate(zeroed_ds['time'].values[select_sweeps]):
    time_ds = zeroed_ds.sel(time=time)
#     bins_time = bins + sweep_avg_ds.sel(time=time, direction='up')['current'].values
#     print(np.mean(bins), np.mean(bins_time))
    hist, bin_edges = np.histogram(time_ds['current'].values, # / sweep_avg_ds.sel(time=time)['current'].values, 
                                   bins='auto', density=True)
#     hist, bin_edges = np.histogram(time_ds['current'].values, bins='auto', density=True)
    bins = (bin_edges[:-1] + bin_edges[1:]) / 2
#     axes[i].plot(bins, hist, label=f"V={sweep_avg_ds.sel(time=time).mean('direction')['voltage'].values}")
    ax[i+1].plot(bins, hist, '.', color='silver', mfc='none', alpha=0.7,
                 label=r"V={:.3g}, std = {:.3g}".format(sweep_avg_ds.sel(time=time)['voltage'].values, sweep_avg_ds.sel(time=time)['d_current'].values))

    fit_data = gaussian.fit(bins, hist) 
    
    ax[i+1].plot(*fit_data.get_fit_plottables(), color=probe_colours[probe],
                 label=r'Gaussian fit - $\sigma$ = {:.3g}'.format(fit_data.get_param("sigma")))
    
    
#     time_ds = zeroed_ds.sel(time=time, direction='down')
#     hist = time_ds['current'].plot.hist(bins=bins)
ax[1].set_ylabel('Normalised Count [arb]')
ax[1].set_xlabel(r'$I_{sweep} - \bar{I}$ [A]')
ax[1].legend()

fig.tight_layout()
plt.show()

In [16]:
bins = np.linspace(-0.1,0.1,200)

plt.figure()
for time in zeroed_ds['time'].values:
    direction = 'up'
    time_ds = zeroed_ds.sel(time=time, direction=direction)
#     bins_time = bins + sweep_avg_ds.sel(time=time, direction='up')['current'].values
#     print(np.mean(bins), np.mean(bins_time))
    hist, bin_edges = np.histogram(time_ds['voltage'].values, bins='auto', density=True)
    bins = (bin_edges[:-1] + bin_edges[1:]) / 2
    plt.plot(bins, hist)
#     time_ds = zeroed_ds.sel(time=time, direction='down')
#     hist = time_ds['current'].plot.hist(bins=bins)
    
plt.show()

In [17]:
# Data-IV to TS-IV Comparison

fig, ax = plt.subplots()
probes = lp.MagnumProbes()

sweep_trim_ds = sweep_avg_ds.isel(time=slice(0,33))


ax.errorbar(sweep_trim_ds['voltage'].values, -sweep_trim_ds['current'].values, 
            yerr=sweep_trim_ds['d_current'].values,
            xerr=sweep_trim_ds['d_voltage'].values,
            linestyle='none', color='k', ecolor='k', label='Sweep-averaged IV', zorder=2)

# Plot the whole IV in an inset axis
inner_ax = plt.axes([0.2, 0.35, .2, .2])
(-sweep_avg_ds.set_coords('voltage')['current']).plot(x='voltage', ax=inner_ax)
inner_ax.set_title('Whole IV')
inner_ax.set_xlabel('V')
inner_ax.set_ylabel('I')
inner_ax.set_xticks([])
inner_ax.set_yticks([])

shot_iv = iv.IVData(sweep_avg_ds['voltage'].values,
                    -sweep_avg_ds['current'].values,
                    sweep_avg_ds['shot_time'].values,
                    sigma=sweep_avg_ds['d_current'].values)

fitter = fts.FullIVFitter()
shot_fit = shot_iv.multi_fit(sat_region=-52)
fit_dens = probes.probe_s.get_density(shot_fit.get_isat(), shot_fit.get_temp(), alpha=np.radians(7.98))[0]
print(fit_dens)

temp = metadata_ds.ts_temp_max.values
dens = metadata_ds.ts_dens_max.values

chi_2_str = r"$\chi^2_{red}$"

ax.plot(*shot_fit.get_fit_plottables(), label=f'Fit - T_e={shot_fit.get_temp():.3g}, n_e={fit_dens:.3g}, {chi_2_str} = {shot_fit.reduced_chi2:.3g}')
# ax.plot(shot_fit.raw_x, 
#         probes.probe_s.get_analytical_iv(
#             shot_fit.raw_x, 
#             shot_fit.get_floating_pot(), 
#             shot_fit.get_sheath_exp(),
#             temp,
#             dens
#         ),
#         label=f'TS - T_e={temp:.3g}, n_e={dens:.3g}')


ax.legend()

1.2525741624238445e+19


## $I_{sat}$ variance over the course of a shot

This section examines the distribution of fitted $I_{sat}$ values for each individual sweep and then deconvoluting that Gaussian from the sweep-averaged distribution to see what the resulting individual measurement error/distribution is. 

In [14]:
sat_region = -52

siv_fitter = fts.StraightIVFitter()
siv_fitter.set_fixed_values({'T_e':metadata_ds.ts_temp_max.values})

sl_fitter = fts.StraightLineFitter()

In [15]:
fig, ax = plt.subplots(2)

for sweep in shot_ds.sweep.values[0:1]:
    for d in ['up', 'down']:
        ds = shot_ds.sel(sweep=sweep, direction=d)
        ds = ds.set_coords('voltage').where(ds.voltage < sat_region, drop=True)
        ds['current'].plot.line(x='voltage', ax=ax[0])
        fit_data = siv_fitter.fit(ds['voltage'].values, ds['current'].values)
        sl_fit = sl_fitter.fit(ds['voltage'].values, ds['current'].values)
        ax[0].plot(*fit_data.get_fit_plottables(), label=fit_data.get_param_str())
        ax[0].plot(*sl_fit.get_fit_plottables(), label=sl_fit.get_param_str())
    ax[0].legend()
        
    mean_ds = shot_ds.sel(sweep=sweep).mean('direction')
    mean_ds = mean_ds.set_coords('voltage').where(mean_ds.voltage < sat_region, drop=True)
    mean_ds['current'].plot.line(x='voltage', ax=ax[1])
    fit_data = siv_fitter.fit(mean_ds['voltage'].values, mean_ds['current'].values)
    ax[1].plot(*fit_data.get_fit_plottables(), label=fit_data.get_param_str())
    ax[1].legend()

In [44]:
densities = np.zeros_like(shot_ds.sweep.values)
isats = []
theta = np.arange(1,10)
sheath_exps = magnum_probes['S'].get_sheath_exp_param(metadata_ds.ts_temp_max.values, metadata_ds.ts_dens_max.values, np.radians(theta))
plt.figure()
plt.plot(theta, sheath_exps)

In [45]:
a = magnum_probes['S'].get_sheath_exp_param(metadata_ds.ts_temp_max.values, metadata_ds.ts_dens_max.values, np.radians(metadata_ds.shot_tilt.values))

In [46]:
# fig, ax = plt.subplots(2)

for sweep in shot_ds.sweep.values:
    mean_ds = shot_ds.sel(sweep=sweep).mean('direction')
    mean_ds = mean_ds.set_coords('voltage').where(mean_ds.voltage < sat_region, drop=True)
#     mean_ds['current'].plot.line(x='voltage', ax=ax[1])
    try:
        fit_data = siv_fitter.fit(mean_ds['voltage'].values, mean_ds['current'].values)
        isats.append(-fit_data.get_param('I_sat'))
        dens = magnum_probes['S'].get_density(-fit_data.get_param('I_sat'), metadata_ds.ts_temp_max.values, metadata_ds.shot_tilt.values)
        densities[sweep] = dens
    except RuntimeError:
        densities[sweep] = 0.0
    
densities

array([6142134017749929984, 6019560517454835712, 5968978580960777216, ...,
       6919039551277216768, 6825227483630472192, 6412807599827957760])

In [47]:
fig, ax = plt.subplots()

dens_no_zeros = densities[np.where(densities != 0.0)]

# hist, bin_edges = np.histogram(-dens_no_zeros, bins='auto', density=True)
hist, bin_edges = np.histogram(isats, bins='auto', density=True)
bins = (bin_edges[:-1] + bin_edges[1:]) / 2
ax.plot(bins, hist)

g_fit = gaussian.fit(bins, hist) 
ax.plot(*g_fit.get_fit_plottables(), label=r'Gaussian fit - $\sigma$ = {:.3g}'.format(g_fit.get_param("sigma")))
ax.legend()

In [48]:
# Plot looking at deconvoluting the plasma parameter variance and the measurement variance

zeroed_ds = shot_ds - sweep_avg_ds

select_sweeps = [23,-1]

fig, axes = plt.subplots(2, sharex=True)

for i, time in enumerate(zeroed_ds['time'].values[select_sweeps]):
    time_ds = zeroed_ds.sel(time=time)
#     bins_time = bins + sweep_avg_ds.sel(time=time, direction='up')['current'].values
#     print(np.mean(bins), np.mean(bins_time))
    hist, bin_edges = np.histogram(time_ds['current'].values, bins='auto', density=True)
    bins = (bin_edges[:-1] + bin_edges[1:]) / 2
#     axes[i].plot(bins, hist, label=f"V={sweep_avg_ds.sel(time=time).mean('direction')['voltage'].values}")
    axes[i].plot(bins, hist, label=r"V={:.3g}, std = {:.3g}".format(sweep_avg_ds.sel(time=time)['voltage'].values, sweep_avg_ds.sel(time=time)['d_current'].values))

    fit_data = gaussian.fit(bins, hist) 
    
    axes[i].plot(*fit_data.get_fit_plottables(), label=r'Gaussian fit - $\sigma$ = {:.3g}'.format(fit_data.get_param("sigma")))
    
    deconvolved_params = [np.sqrt(fit_data.get_param("sigma")**2 - g_fit.get_param("sigma")**2), 0]# fit_data.get_param("x_0") - g_fit.get_param("x_0")]
    
    new_gaussian = gaussian.fit_function(bins, *deconvolved_params)
    axes[i].plot(bins, new_gaussian, label=r'Deconvolved Gaussian - $\sigma$ = {:.3g}'.format(deconvolved_params[0]))
    
    
#     time_ds = zeroed_ds.sel(time=time, direction='down')
#     hist = time_ds['current'].plot.hist(bins=bins)
axes[0].legend()
axes[1].legend()
plt.show()

# Single Bias Distribution Analysis

In [26]:
# Get row from shot_number instead of ADC index
single_bias = analysed_infos_df.loc[analysed_infos_df['shot_number'] == single_bias_shot]
single_bias['filename'].values

array(['analysed_4_downsampled/a112_117_6696468713881848832.nc'],
      dtype=object)

In [27]:
single_bias_ds = xr.open_dataset(single_bias['filename'].values[0])
sb_metadata_ds = xr.open_dataset('all_meta_data.nc').sel(shot_number=single_bias_shot).load()
print(single_bias_ds)
metadata_ds

<xarray.Dataset>
Dimensions:     (direction: 2, probe: 2, sweep: 193, time: 250)
Coordinates:
  * time        (time) float64 0.0 0.0001 0.0002 0.0003 ... 0.0247 0.0248 0.0249
  * direction   (direction) object 'up' 'down'
  * probe       (probe) object 'S' 'L'
Dimensions without coordinates: sweep
Data variables:
    voltage     (probe, direction, sweep, time) float64 ...
    current     (probe, direction, sweep, time) float64 ...
    shot_time   (probe, direction, sweep, time) float64 ...
    start_time  (probe, direction, sweep) float64 ...


<xarray.Dataset>
Dimensions:                 (ts_radial_pos: 46)
Coordinates:
    shot_number             int32 337
    ts_number               float64 404.0
    ts_timestamp            float64 6.699e+18
    ts_time                 datetime64[ns] 2019-06-05T15:11:12.409731
    adc_index               float64 329.0
    adc_time                datetime64[ns] 2019-06-05T15:11:12
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            object '2019-06-05 16h 11m 34s TT_06699048922136188177.adc'
    ts_density              (ts_radial_pos) float64 nan nan nan ... nan nan nan
    ts_temperature          (ts_radial_pos) float64 nan nan nan ... nan nan nan
    ts_d_density            (ts_radial_pos) float64 nan nan nan ... nan nan nan
    ts_d_temperature        (ts_radial_pos) float64 nan nan nan ... nan nan nan
    adc_folder              object '2019-06-05_Leland/'
    adc_calibration_index   object '272'
    adc_4_probe    

In [28]:
sb_shot_ds = single_bias_ds.sel(probe=probe)

In [29]:
t_0 = 0.0
fig, ax = plt.subplots()

for sweep in sb_shot_ds.sweep.values:
    sweep_ds = sb_shot_ds.sel(sweep=sweep)
    for direction in sweep_ds.direction.values:
        plot_ds = sweep_ds.sel(direction=direction)
        plot_ds['time'] += t_0
        t_0 += plot_ds['time'].size
        plot_ds['current'].plot(ax=ax)

In [30]:
time_series_ds = sb_shot_ds.stack(shot_time=('sweep', 'direction', 'time')).reset_index(['shot_time'])
time_series_ds = time_series_ds.assign({'density': magnum_probes['S'].get_density(-time_series_ds['current'], 4.749, alpha=np.radians(7.98))})
time_series_ds

<xarray.Dataset>
Dimensions:     (shot_time: 96500)
Coordinates:
    probe       <U1 'S'
    sweep       (shot_time) int64 0 0 0 0 0 0 0 ... 192 192 192 192 192 192 192
    direction   (shot_time) object 'up' 'up' 'up' 'up' ... 'down' 'down' 'down'
    time        (shot_time) float64 1.858e+07 1.858e+07 ... 1.858e+07 1.858e+07
Dimensions without coordinates: shot_time
Data variables:
    voltage     (shot_time) float64 -101.9 -101.9 -101.9 ... -101.8 -101.9
    current     (shot_time) float64 -0.02888 -0.02629 ... -0.02801 -0.02629
    start_time  (shot_time) float64 1.297 1.297 1.297 ... 10.92 10.92 10.92
    density     (shot_time) float64 2.606e+18 2.373e+18 ... 2.528e+18 2.373e+18

In [31]:
time_series_ds['current'].plot()
time_series_ds.mean(['shot_time'])['current'].values

array(-0.02648178)

In [32]:
fig, ax = plt.subplots()

auto_hist, auto_bin_edges = np.histogram(time_series_ds['current'].values, bins='auto', density=True)
auto_bins = (auto_bin_edges[:-1] + auto_bin_edges[1:]) / 2

manual_hist, manual_bin_edges = np.histogram(time_series_ds['current'].values, bins=22, density=True)
manual_bins = (manual_bin_edges[:-1] + manual_bin_edges[1:]) / 2

auto_fit_data = gaussian.fit(auto_bins, auto_hist)
man_fit_data = gaussian.fit(manual_bins, manual_hist)

gaussian_str = r'{} - $\sigma = $ {:.3g}, $\mu = $ {:.3g}'

ax.plot(auto_bins, auto_hist, label='dist - auto')
ax.plot(manual_bins, manual_hist, label='dist - manual')
ax.plot(*auto_fit_data.get_fit_plottables(), label=gaussian_str.format('auto', auto_fit_data.get_param("sigma"), auto_fit_data.get_param("x_0")))
ax.plot(*man_fit_data.get_fit_plottables(), label=gaussian_str.format('manual', man_fit_data.get_param("sigma"), man_fit_data.get_param("x_0")))

ax.axvline(x=time_series_ds.mean(['shot_time'])['current'].values, **c.AX_LINE_DEFAULTS)

for bin_edge in auto_bin_edges:
    ax.axvline(x=bin_edge, linestyle='dotted', color='silver', linewidth=0.5)
for bin_edge in manual_bin_edges:
    ax.axvline(x=bin_edge, linestyle=':', color='darkgrey', linewidth=0.5)
    
ax.legend()

In [120]:
same_param_ds

<xarray.Dataset>
Dimensions:                 (shot_number: 23, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int32 328 329 330 331 ... 351 352 353
    ts_number               (shot_number) float64 377.0 380.0 ... 449.0 453.0
    ts_timestamp            (shot_number) float64 6.699e+18 ... 6.699e+18
    ts_time                 (shot_number) datetime64[ns] 2019-06-05T14:47:50.700290 ... 2019-06-05T16:03:37.541035
    adc_index               (shot_number) float64 320.0 321.0 ... 345.0 346.0
    adc_time                (shot_number) datetime64[ns] 2019-06-05T14:47:50 ... 2019-06-05T16:03:37
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            (shot_number) object '2019-06-05 15h 48m 07s TT_06699042901839982997.adc' ... '2019-06-05 17h 03m 59s TT_06699062430372281696.adc'
    ts_density              (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_temperature          (shot_number, 

In [135]:
fig, ax = plt.subplots()
for tsrp in same_param_ds.ts_radial_pos.values[20:25]:
    ds = same_param_ds.sel(ts_radial_pos=tsrp)
    mean_dens = ds.mean('shot_number').ts_density.values
    hist, bin_edges = np.histogram((ds.ts_density.values - mean_dens) / mean_dens, bins='auto', density=True)
    bins = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    ax.plot(bins, hist)
    

In [126]:
fig, ax = plt.subplots()

same_param_ds.ts_density.plot.hist(hue='ts_radial_pos')

## Thomson Distribution Analysis

Looking at the distribution of temperatures and densities for the same magnum source input parameters - the idea being to get a measure of the spread of plasma parameters and then deconvolve that from the sweep-averaged distribution to get the measurement error/distribution. 

In [12]:
rounded_md_ds = all_md_ds.copy(deep=True)
rounded_md_ds['shot_b_field'] = all_md_ds.shot_b_field.round(1)
rounded_md_ds['shot_source_current'] = (all_md_ds.shot_source_current / 5).round() * 5
rounded_md_ds['shot_hydrogen_gf'] = (2 * all_md_ds.shot_hydrogen_gf).round() / 2
rounded_md_ds['shot_helium_gf'] = (2 * all_md_ds.shot_helium_gf).round() / 2
rounded_md_ds['shot_deuterium_gf'] = (2 * all_md_ds.shot_deuterium_gf).round() / 2

rounded_md_ds #.shot_source_current.values

<xarray.Dataset>
Dimensions:                 (shot_number: 523, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int32 0 1 2 3 4 ... 519 520 521 522
    ts_number               (shot_number) float64 nan nan nan ... 625.0 626.0
    ts_timestamp            (shot_number) float64 nan nan ... 6.7e+18 6.7e+18
    ts_time                 (shot_number) datetime64[ns] NaT ... 2019-06-07T15:10:32.317018
    adc_index               (shot_number) float64 nan nan nan ... 525.0 526.0
    adc_time                (shot_number) datetime64[ns] NaT ... 2019-06-07T15:10:30
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            (shot_number) object '' ... '2019-06-07 16h 10m 40s TT_06699790911698096530.adc'
    ts_density              (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_temperature          (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_d_density            (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_d_temperature        (shot_number, ts_radial_pos) float64 nan nan ... nan
    adc_folder              (shot_number) object '' '' ... '2019-06-07_Leland/'
    adc_calibration_index   (shot_number) object '' '' '' ... '502' '502' '502'
    adc_4_probe             (shot_number) object '' '' '' 'S' ... 'S' 'S' 'S'
    adc_5_probe             (shot_number) object '' '' '' 'L' ... 'L' 'L' 'L'
    ts_temp_max             (shot_number) float64 nan nan nan ... 1.365 1.151
    ts_dens_max             (shot_number) float64 nan nan ... 3.083e+20
    shot_end_time           (shot_number) datetime64[ns] 2019-05-28T13:15:01 ... 2019-06-07T15:10:41
    ts_fl                   (shot_number) bool False False False ... True True
    shot_time               (shot_number) datetime64[ns] 2019-05-28T13:13:57 ... 2019-06-07T15:10:25
    shot_source_current     (shot_number) float64 105.0 120.0 ... 135.0 135.0
    shot_b_field            (shot_number) float64 0.0 0.1 0.2 ... 1.5 1.5 1.5
    shot_tilt               (shot_number) float64 -10.0 -10.0 ... 7.98 7.98
    shot_target_pos         (shot_number) float64 -0.1119 -0.1119 ... -0.127
    shot_target_rot         (shot_number) float64 -89.94 -89.94 ... -89.94
    shot_target_pumpspeed   (shot_number) int32 82 82 82 82 82 ... 25 25 25 25
    shot_source_pumpspeed   (shot_number) int32 82 82 82 82 82 ... 82 82 82 82
    shot_heating_pumpspeed  (shot_number) int32 82 82 82 82 82 ... 82 82 82 82
    shot_source_pos         (shot_number) float64 1.35 1.35 1.35 ... 1.2 1.2 1.2
    shot_deuterium_gf       (shot_number) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    shot_hydrogen_gf        (shot_number) float64 0.0 10.0 10.0 ... 9.5 9.5 9.5
    shot_helium_gf          (shot_number) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    shot_hydrogen_puffed    (shot_number) float64 -0.03709 -0.03308 ... 6.755
    shot_helium_puffed      (shot_number) float64 -0.05415 -0.04697 ... -0.05601
    adc_timestamp           (shot_number) float64 nan nan ... 6.7e+18 6.7e+18
    adc_4_coax              (shot_number) float64 nan nan nan ... 3.0 3.0 3.0
    adc_4_shunt_resistance  (shot_number) float64 nan nan nan ... 1.0 1.0 1.0
    adc_5_coax              (shot_number) float64 nan nan nan ... 4.0 4.0 4.0
    adc_5_shunt_resistance  (shot_number) float64 nan nan nan ... 1.0 1.0 1.0
    adc_calibration_fl      (shot_number) float64 nan nan nan ... 0.0 0.0 0.0
    shot_fl                 (shot_number) float64 nan nan nan ... 1.0 1.0 1.0
    acquisition_length      (shot_number) float64 nan nan nan ... 10.0 10.0 10.0
    adc_end_time            (shot_number) datetime64[ns] NaT ... 2019-06-07T15:10:40
    adc_end_timestamp       (shot_number) float64 nan nan ... 6.7e+18 6.7e+18
    adc_freqs               (shot_number) float64 nan nan nan ... 1e+06 1e+06

In [13]:
input_groups = rounded_md_ds.mean('ts_radial_pos').to_dataframe().groupby(['shot_b_field', 'shot_source_current', 'shot_hydrogen_gf']).groups

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [14]:
for input_group in input_groups:
    input_group_size = len(input_groups[input_group])
    if input_group_size > 5:
        print(input_group, input_group_size)

(0.5, 130.0, 12.0) 27
(0.8, 100.0, 7.5) 47
(0.8, 100.0, 8.5) 57
(0.8, 120.0, 0.0) 28
(0.8, 150.0, 8.5) 26
(0.8, 180.0, 0.0) 24
(0.8, 180.0, 7.5) 10
(1.2, 100.0, 0.0) 31
(1.2, 100.0, 8.5) 27
(1.2, 110.0, 0.0) 13
(1.2, 110.0, 8.5) 23
(1.2, 160.0, 0.0) 6
(1.2, 200.0, 0.0) 7
(1.5, 120.0, 0.0) 7
(1.5, 130.0, 9.5) 9
(1.5, 135.0, 9.5) 49


Going for (1.5, 135.0, 9.5) as this is the highest group size after the axial scan, for which there are not many thomson scattering shots. 

In [15]:
settings_oi = input_groups[(0.8, 100.0, 7.5)].drop(282)
settings_oi

Int64Index([ 66,  70,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
             83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
             96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
            109, 110, 111, 112, 283, 308, 309],
           dtype='int64', name='shot_number')

In [16]:
ts_data_oi = all_ts_ds.where(all_ts_ds.shot_number.isin(settings_oi), drop=True)
ts_data_oi = ts_data_oi.where(np.logical_and(np.isfinite(ts_data_oi.ts_density), np.isfinite(ts_data_oi.ts_temperature)), drop=True)
ts_data_oi

<xarray.Dataset>
Dimensions:           (ts_number: 121, ts_radial_pos: 38)
Coordinates:
  * ts_radial_pos     (ts_radial_pos) float64 -28.11 -24.76 ... 33.8 35.47
  * ts_number         (ts_number) int64 62 63 64 68 70 ... 182 183 315 340 341
    ts_timestamp      (ts_number) int64 6696424748853370885 ... 6699006291123691866
    ts_time           (ts_number) datetime64[ns] 2019-05-29T13:28:04.455233 ... 2019-06-05T12:25:46.603651
    shot_number       (ts_number) int64 66 66 66 70 72 ... 112 112 283 308 309
Data variables:
    ts_density        (ts_number, ts_radial_pos) float64 nan nan nan ... nan nan
    ts_d_density      (ts_number, ts_radial_pos) float64 nan nan nan ... nan nan
    ts_temperature    (ts_number, ts_radial_pos) float64 nan nan nan ... nan nan
    ts_d_temperature  (ts_number, ts_radial_pos) float64 nan nan nan ... nan nan

In [17]:
rounded_md_ds.sel(shot_number=ts_data_oi.shot_number.values).shot_target_pos.plot(marker='x')

In [18]:
fig, ax = plt.subplots()
_ = ts_data_oi.ts_density.plot.line(hue='ts_number', ax=ax)

In [158]:
fig, ax = plt.subplots()
ts_data_oi.ts_density.plot(ax=ax)

In [159]:
ts_data_oi = ts_data_oi.assign({
    'mean_density': ts_data_oi.mean('ts_number').ts_density, 
    'mean_temperature': ts_data_oi.mean('ts_number').ts_temperature,
    'std_density': ts_data_oi.std('ts_number').ts_density, 
    'std_temperature': ts_data_oi.std('ts_number').ts_temperature,
}) 

In [162]:
fig, ax = plt.subplots(2, sharex=True)

peak_slc = slice(14,16)

ts_data_oi.ts_density.plot(ax=ax[0], hue='shot_number', color='silver')
ts_data_oi.mean_density.plot(ax=ax[0])
ts_data_oi.mean_density.isel(ts_radial_pos=peak_slc).plot(ax=ax[0], color='red')

ts_data_oi.ts_temperature.plot(ax=ax[1], hue='shot_number', color='silver')
ts_data_oi.mean_temperature.plot(ax=ax[1])
ts_data_oi.mean_temperature.isel(ts_radial_pos=peak_slc).plot(ax=ax[1], color='red')


In [173]:
fig, ax = plt.subplots(2, sharex=True)

(ts_data_oi.ts_density - ts_data_oi.mean_density).plot.line(hue='shot_number', ax=ax[0])
(ts_data_oi.ts_density - ts_data_oi.mean_density).isel(ts_radial_pos=peak_slc).plot.line(hue='shot_number', ax=ax[0], color='black')

(ts_data_oi.ts_temperature - ts_data_oi.mean_temperature).plot.line(hue='shot_number', ax=ax[1])
_ = (ts_data_oi.ts_temperature - ts_data_oi.mean_temperature).isel(ts_radial_pos=peak_slc).plot.line(hue='shot_number', ax=ax[1], color='black')

In [187]:
dens_peak = (ts_data_oi.ts_density - ts_data_oi.mean_density).isel(ts_radial_pos=peak_slc)
temp_peak = (ts_data_oi.ts_temperature - ts_data_oi.mean_temperature).isel(ts_radial_pos=peak_slc)

In [199]:
# rounded_md_ds.where(rounded_md_ds.shot_number.isin(settings_oi), drop=True)
importlib.reload(lp)
magnum_probes = lp.MagnumProbes()
isat = magnum_probes[probe].get_isat(temp_peak, dens_peak, np.radians(8)).dropna('ts_number')

In [202]:
fig, ax = plt.subplots()
hist, bin_edges = np.histogram(isat.values, density=True)
bins = (bin_edges[:-1] + bin_edges[1:]) / 2
ax.plot(bins, hist)

fit_data = gaussian.fit(bins, hist) 
ax.plot(*fit_data.get_fit_plottables(), label=r'Gaussian fit - $\sigma$ = {:.3g}'.format(fit_data.get_param("sigma")))

ax.legend()

Looks like a very small variation ($\sigma_{TS} = 0.006$ compared to $\sigma_{shot} \sim 0.01$) so this can probably be ignored in comarison to the variation in voltage signal. 

---

## All Normalised TS Shots Distribution
Small section looking at whether all TS shots can be compared (when normalised).

N.B. On second thought I'm not sure how exactly to do this, how to renormalise afterwards?

### First up: Normalising one set of input parameters

In [1]:
normalised_dens = (ts_data_oi.ts_density - ts_data_oi.mean_density) / ts_data_oi.mean_density
normalised_temp = (ts_data_oi.ts_temperature - ts_data_oi.mean_temperature) / ts_data_oi.mean_temperature


fig, ax = plt.subplots()
_ = normalised_dens.plot.line(ax=ax, hue='ts_number')

NameError: name 'ts_data_oi' is not defined

In [224]:
all_normalised_dens = normalised_dens.values.reshape(121*38)
all_normalised_dens = all_normalised_dens[np.isfinite(all_normalised_dens)]

all_normalised_temp = normalised_temp.values.reshape(121*38)
all_normalised_temp = all_normalised_temp[np.isfinite(all_normalised_temp)]
all_normalised_temp.shape

(3720,)

In [230]:
print(121*38)
normalised_dens.size


4598


4598

In [223]:
normalised_isat = magnum_probes[probe].get_isat(all_normalised_temp, all_normalised_dens, np.radians(8))

/home/jleland/coding/projects/flopter/flopter/core/lputils.py:210: RuntimeWarning: invalid value encountered in sqrt
  c.PROTON_MASS * mass))


In [226]:
fig, ax = plt.subplots()
hist, bin_edges = np.histogram(all_normalised_dens, bins='auto', density=True)
bins = (bin_edges[:-1] + bin_edges[1:]) / 2
ax.plot(bins, hist)

fit_data = gaussian.fit(bins, hist) 
ax.plot(*fit_data.get_fit_plottables(), label=r'Gaussian fit - $\sigma$ = {:.3g}'.format(fit_data.get_param("sigma")))

ax.legend()

### Now apply to all of them

In [232]:
all_norm_denss = []
all_norm_temps = []

for input_group in input_groups.values():
    ig_ts_data = all_ts_ds.where(all_ts_ds.shot_number.isin(input_group), drop=True)
    ig_ts_data = ig_ts_data.where(np.logical_and(
        np.isfinite(ig_ts_data.ts_density), 
        np.isfinite(ig_ts_data.ts_temperature)
    ), drop=True)
    
    ig_ts_data = ig_ts_data.assign({
        'mean_density': ig_ts_data.mean('ts_number').ts_density, 
        'mean_temperature': ig_ts_data.mean('ts_number').ts_temperature,
        'std_density': ig_ts_data.std('ts_number').ts_density, 
        'std_temperature': ig_ts_data.std('ts_number').ts_temperature,
    })
    
    ig_normalised_dens = (ig_ts_data.ts_density - ig_ts_data.mean_density) / ig_ts_data.mean_density
    ig_normalised_temp = (ig_ts_data.ts_temperature - ig_ts_data.mean_temperature) / ig_ts_data.mean_temperature
    
    all_ig_norm_dens = ig_normalised_dens.values.reshape(ig_normalised_dens.size)
    all_ig_norm_dens = all_ig_norm_dens[np.isfinite(all_ig_norm_dens)]
    
    all_ig_norm_temp = ig_normalised_temp.values.reshape(ig_normalised_temp.size)
    all_ig_norm_temp = all_ig_norm_temp[np.isfinite(all_ig_norm_temp)]
    
    all_norm_denss.append(all_ig_norm_dens)
    all_norm_temps.append(all_ig_norm_temp)

In [236]:
all_norm_dens = np.concatenate(all_norm_denss)
all_norm_temp = np.concatenate(all_norm_temps)

In [238]:
def plot_and_fit_gaussian(data):
    fig, ax = plt.subplots()
    hist, bin_edges = np.histogram(data, bins='auto', density=True)
    bins = (bin_edges[:-1] + bin_edges[1:]) / 2
    ax.plot(bins, hist)

    fit_data = gaussian.fit(bins, hist) 
    ax.plot(*fit_data.get_fit_plottables(), label=r'Gaussian fit - $\sigma$ = {:.3g}'.format(fit_data.get_param("sigma")))

    ax.legend()

In [240]:
plot_and_fit_gaussian(all_norm_temp)

## Checking The Effect of Probe Positioning on TS Profiles

In [19]:
for input_group in input_groups:
    input_group_size = len(input_groups[input_group])
    if input_group_size > 5:
        print(input_group, input_group_size)

(0.5, 130.0, 12.0) 27
(0.8, 100.0, 7.5) 47
(0.8, 100.0, 8.5) 57
(0.8, 120.0, 0.0) 28
(0.8, 150.0, 8.5) 26
(0.8, 180.0, 0.0) 24
(0.8, 180.0, 7.5) 10
(1.2, 100.0, 0.0) 31
(1.2, 100.0, 8.5) 27
(1.2, 110.0, 0.0) 13
(1.2, 110.0, 8.5) 23
(1.2, 160.0, 0.0) 6
(1.2, 200.0, 0.0) 7
(1.5, 120.0, 0.0) 7
(1.5, 130.0, 9.5) 9
(1.5, 135.0, 9.5) 49


In [20]:
fig, ax = plt.subplots()

for input_group in input_groups:
    input_group_size = len(input_groups[input_group])
    if input_group_size < 2:
        continue
        
    settings_oi = input_groups[input_group]
    ts_data_oi = all_ts_ds.where(all_ts_ds.shot_number.isin(settings_oi), drop=True)
    ts_data_oi = ts_data_oi.where(np.logical_and(np.isfinite(ts_data_oi.ts_density), 
                                                 np.isfinite(ts_data_oi.ts_temperature)), drop=True)
    
    meta_data_oi = rounded_md_ds.sel(shot_number=ts_data_oi.shot_number.values)
    linear_pos_count = meta_data_oi.groupby('shot_target_pos').count().shot_target_pos.size
    
    if linear_pos_count < 2:
        continue
        
    print(input_group, input_group_size, linear_pos_count)
    meta_data_oi.shot_target_pos.plot.line(x='shot_number', marker='x', ax=ax)

(0.2, 150.0, 10.0) 4 2
(0.5, 130.0, 12.0) 27 2
(0.8, 100.0, 0.0) 3 2
(0.8, 100.0, 7.5) 47 2
(0.8, 100.0, 8.5) 57 3
(0.8, 120.0, 0.0) 28 2
(0.8, 180.0, 0.0) 24 2
(1.2, 100.0, 8.5) 27 3
(1.2, 130.0, 8.5) 3 2
(1.2, 150.0, 8.5) 3 2
(1.5, 135.0, 9.5) 49 2


In [21]:
settings_oi = input_groups[
#     (0.8, 100.0, 8.5)
    (1.2, 100.0, 8.5)
]
ts_data_oi = all_ts_ds.where(all_ts_ds.shot_number.isin(settings_oi), drop=True)
# ts_data_oi = ts_data_oi.where(np.logical_and(np.isfinite(ts_data_oi.ts_density), 
#                                              np.isfinite(ts_data_oi.ts_temperature)), drop=True)

fig, ax = plt.subplots()
meta_data_oi = rounded_md_ds.sel(shot_number=ts_data_oi.shot_number.values)
meta_data_oi.shot_target_pos.plot.line(x='shot_number', marker='x', ax=ax)

grouped_ts = meta_data_oi.groupby('shot_target_pos').mean()
grouped_ts

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.Dataset>
Dimensions:                 (shot_target_pos: 3, ts_radial_pos: 46)
Coordinates:
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
  * shot_target_pos         (shot_target_pos) float64 -0.2 -0.17 -0.15
Data variables:
    ts_density              (shot_target_pos, ts_radial_pos) float64 nan ... nan
    ts_temperature          (shot_target_pos, ts_radial_pos) float64 nan ... nan
    ts_d_density            (shot_target_pos, ts_radial_pos) float64 nan ... nan
    ts_d_temperature        (shot_target_pos, ts_radial_pos) float64 nan ... nan
    ts_temp_max             (shot_target_pos) float64 2.463 3.115 3.168
    ts_dens_max             (shot_target_pos) float64 3.532e+19 ... 1.624e+19
    ts_fl                   (shot_target_pos) float64 1.0 1.0 1.0
    shot_source_current     (shot_target_pos) float64 100.0 100.0 100.0
    shot_b_field            (shot_target_pos) float64 1.2 1.2 1.2
    shot_tilt               (shot_target_pos) float64 -7.504 -0.0355 -0.384
    shot_target_rot         (shot_target_pos) float64 -89.93 -89.94 -89.94
    shot_target_pumpspeed   (shot_target_pos) float64 82.0 82.0 82.0
    shot_source_pumpspeed   (shot_target_pos) float64 82.0 82.0 82.0
    shot_heating_pumpspeed  (shot_target_pos) float64 82.0 82.0 82.0
    shot_source_pos         (shot_target_pos) float64 1.35 1.35 1.282
    shot_deuterium_gf       (shot_target_pos) float64 0.0 0.0 0.0
    shot_hydrogen_gf        (shot_target_pos) float64 8.5 8.5 8.5
    shot_helium_gf          (shot_target_pos) float64 0.0 0.0 0.0
    shot_hydrogen_puffed    (shot_target_pos) float64 6.32 -0.0106 1.517
    shot_helium_puffed      (shot_target_pos) float64 -0.05158 -0.05354 -0.0522
    adc_timestamp           (shot_target_pos) float64 6.698e+18 ... 6.699e+18
    adc_4_coax              (shot_target_pos) float64 1.0 1.0 1.0
    adc_4_shunt_resistance  (shot_target_pos) float64 10.0 1.0 1.0
    adc_5_coax              (shot_target_pos) float64 2.0 2.0 2.0
    adc_5_shunt_resistance  (shot_target_pos) float64 10.0 1.0 1.0
    adc_calibration_fl      (shot_target_pos) float64 0.0 0.0 0.0
    shot_fl                 (shot_target_pos) float64 1.0 1.0 1.0
    acquisition_length      (shot_target_pos) float64 8.5 8.0 12.45
    adc_end_timestamp       (shot_target_pos) float64 6.698e+18 ... 6.699e+18
    adc_freqs               (shot_target_pos) float64 8.75e+05 1e+06 1e+06

In [24]:
fig, ax = plt.subplots(2)
grouped_ts.ts_temperature.plot.line(hue='shot_target_pos', marker='x', ax=ax[0])
grouped_ts.ts_density.plot.line(hue='shot_target_pos', marker='x', ax=ax[1])

In [23]:
fig, ax = plt.subplots()
rounded_md_ds.sel(shot_number=slice(283,318)).shot_target_pos.plot.line(x='shot_number', marker='x')
rounded_md_ds.sel(shot_number=slice(119,142)).shot_target_pos.plot.line(x='shot_number', marker='x')
